In [ ]:
import json
import pprint

with open("Chart4.mdj", 'r', encoding='utf-8') as file:
    data = json.load(file)

data = data["ownedElements"]

for item in data:
    if isinstance(item, dict) and "ownedElements" in item:
      data = item["ownedElements"]

data = data[1]
data = data["regions"][0]

pp = pprint.PrettyPrinter(indent=4)
#pp.pprint(data)

states = dict()
for state in data["vertices"]:
  if "name" in state.keys():
    states[state["_id"]] = state["name"]
  elif "kind" in state.keys():
    states[state["_id"]] = state["kind"]
  else:
    states[state["_id"]] = "final"

transitions = []
for transition in data["transitions"]:
  trans_item = dict()
  #trans_item["id"] = transition["_id"]
  trans_item["source"] = states[transition["source"]["$ref"]]
  trans_item["target"] = states[transition["target"]["$ref"]]

  if "guard" in transition.keys():
    trans_item["guard"] = transition["guard"]

  if "triggers" in transition.keys():
    trans_item["trigger"] = transition["triggers"][0]["name"]

  transitions.append(trans_item)

'''
print("States\n--------------------------------------")
for id in states:
  print(states[id])

print("\n\nTransitions\n--------------------------------------")
for transition in transitions:
  print(transition)
'''

code = ""

imports = """import lejos.hardware.motor.EV3LargeRegulatedMotor;
import lejos.hardware.port.MotorPort;
import lejos.hardware.port.SensorPort;
import lejos.robotics.Color;
import lejos.robotics.RegulatedMotor;
import lejos.robotics.SampleProvider;
import lejos.hardware.sensor.EV3UltrasonicSensor;
import lejos.hardware.sensor.EV3ColorSensor;
import lejos.hardware.Button;
"""

code += imports

control_class = """
public class Rover {
"""

mechanisms = """
\t// Setup Motor Mechanisms
\tpublic static RegulatedMotor m1 = new EV3LargeRegulatedMotor(MotorPort.A);
\tpublic static RegulatedMotor m2 = new EV3LargeRegulatedMotor(MotorPort.B);
\t// Setup Proximity Sensing Mechanism
\tpublic static EV3UltrasonicSensor u = new EV3UltrasonicSensor(SensorPort.S1);
\t// Setup Color Sensing Mechanism
\tpublic static EV3ColorSensor c = new EV3ColorSensor(SensorPort.S2);
"""

main_function = """
\tpublic static void main(String args[]) throws InterruptedException {
"""
code += control_class
code += mechanisms
code += main_function

# Trigger distance
distance_trigger = """
\t/**
\t*
\t* The method gives distance of proximity sensor
\t*
\t* @return the distance from an object in meters
\t*/
\tpublic static float getDistance() {
\t\tfloat t_distance = 0.0f;

\t\tfinal SampleProvider dist = u.getDistanceMode();

\t\tfloat [] sample = new float[dist.sampleSize()];
\t\tdist.fetchSample(sample, 0);
\t\tt_distance = sample[0];

\t\treturn t_distance;
\t}
"""

# Trigger color
color_trigger = """
\t/**
\t* The method returns color of color sensor
\t*
\t* @return and integer value representing the detected color, matching the Color constants
\t*/
\tpublic static int getColor() {
\t\tSampleProvider color = c.getColorIDMode();
\t\tfloat[] sample = new float[color.sampleSize()];

\t\tcolor.fetchSample(sample, 0);

\t\treturn (int)sample[0];
\t}
"""

state_action = dict()

# state_action[STATE] = [entry, do, exit]

# State initial
state_action["initial"] = ["","",""]

# State IDLE
# Stop Moving and Idle
# Wait for Button Trigger
state_action["IDLE"] = ["\t\t\t\tm1.stop();\n\t\t\t\tm2.stop();","",""]

# State FORWARD
# Move Forward at Constant Speed
# Wait for Distance Trigger
# Wait for Color Trigger
state_action["FORWARD"] = ["\t\t\t\tm1.setSpeed(250);\n\t\t\t\tm2.setSpeed(250);", "\t\t\t\t\tm1.forward();\n\t\t\t\t\tm2.forward();", "\t\t\t\t\t\tm1.stop();\n\t\t\t\t\t\tm2.stop();"]

# State BACKWARD
# Move Backward at Constant Speed
# Wait for Timer Trigger
state_action["BACKWARD"] = ["\t\t\t\tm1.setSpeed(250);\n\t\t\t\tm2.setSpeed(250);", "\t\t\t\t\tm1.backward();\n\t\t\t\t\tm2.backward();", "\t\t\t\t\t\tm1.stop();\n\t\t\t\t\t\tm2.stop();"]

# State ROTATE_LEFT
# Rotate Left 90 Degrees
# Transition to State FORWARD
state_action["ROTATE_LEFT"] = ["\t\t\t\tm1.setSpeed(450);\n\t\t\t\tm2.setSpeed(250);\n\t\t\t\tm1.backward();\n\t\t\t\tm2.forward();\n\t\t\t\tThread.sleep(950);\n\t\t\t\tm1.stop();\n\t\t\t\tm2.stop();","",""]

# State ROTATE_RIGHT
# Rotate Right 90 Degrees
# Transition to State FORWARD
state_action["ROTATE_RIGHT"] = ["\t\t\t\tm1.setSpeed(250);\n\t\t\t\tm2.setSpeed(450);\n\t\t\t\tm1.forward();\n\t\t\t\tm2.backward();\n\t\t\t\tThread.sleep(950);\n\t\t\t\tm1.stop();\n\t\t\t\tm2.stop();","",""]

# State final
# End Execution of Program

def stateHandlingText(state):
  sum = ""
  if state == 'final':
    sum += "\t\t\t\tbreak;\n"
    return sum

  if (len(state_action[state][0]) > 0):
    sum += (state_action[state][0])  + '\n' # Entry Action

  sum += ("\t\t\t\twhile(true){\n")

  if (len(state_action[state][1]) > 0):
    sum += (state_action[state][1]) + '\n' # Do Action

  for transition in transitions:
    if (transition['source'] == state):
      if 'guard' in transition.keys():
        if 'time ==' in transition['guard']:
          sum += ("\t\t\t\t\tThread.sleep("+ transition['guard'][8:] +");\n")
          sum += ('\t\t\t\t\tstate = "' + transition['target'] + '";\n')
          if (len(state_action[state][2]) > 0):
            sum += (state_action[state][2]) + '\n' # Exit Action
          sum += ("\t\t\t\t\tbreak;\n")
        else:
          if 'button ==' in transition['guard']:
            sum += ("\t\t\t\t\tif (Button." + transition['guard'][10:] + ".isDown()) {\n")
          else:
            sum += ("\t\t\t\t\tif (" + transition['guard'].replace("color","getColor()").replace("distance","getDistance()") + ") {\n")
          sum += ('\t\t\t\t\t\tstate = "' + transition['target'] + '";\n')
          if (len(state_action[state][2]) > 0):
            sum += (state_action[state][2]) + '\n' # Exit Action
          sum += ("\t\t\t\t\t\tbreak;\n")
          sum += ("\t\t\t\t\t}\n")
      else:
        sum += ('\t\t\t\t\tstate = "' + transition['target'] + '";\n')
        if (len(state_action[state][2]) > 0):
          sum += (state_action[state][2]) + '\n' # Exit Action
        sum += ("\t\t\t\t\tbreak;\n")
  sum += ("\t\t\t\t}\n")
  return sum

code += ('\t\tString state = "initial";\n')
first = True
code += ("\t\twhile(true) {\n")

for id in states:
  if first:
    code += ('\t\t\tif(state == "' + states[id] + '"){\n')
    first = False
  else:
    code +=('\t\t\telse if(state == "' + states[id] + '"){\n')
  code += stateHandlingText(states[id])
  code +=("\t\t\t}\n")

code +=("\t\t}\n")
code +=("\t}\n")

code += (distance_trigger)
code += (color_trigger)
code += ("}")


print(code)



import lejos.hardware.motor.EV3LargeRegulatedMotor;
import lejos.hardware.port.MotorPort;
import lejos.hardware.port.SensorPort;
import lejos.robotics.Color;
import lejos.robotics.RegulatedMotor;
import lejos.robotics.SampleProvider;
import lejos.hardware.sensor.EV3UltrasonicSensor;
import lejos.hardware.sensor.EV3ColorSensor;
import lejos.hardware.Button;

public class Rover {

	// Setup Motor Mechanisms
	public static RegulatedMotor m1 = new EV3LargeRegulatedMotor(MotorPort.A);
	public static RegulatedMotor m2 = new EV3LargeRegulatedMotor(MotorPort.B);
	// Setup Proximity Sensing Mechanism
	public static EV3UltrasonicSensor u = new EV3UltrasonicSensor(SensorPort.S1);
	// Setup Color Sensing Mechanism
	public static EV3ColorSensor c = new EV3ColorSensor(SensorPort.S2);

	public static void main(String args[]) throws InterruptedException {
		String state = "initial";
		while(true) {
			if(state == "initial"){
				while(true){
					state = "IDLE";
					break;
				}
			}
			else if(state

# New Section